In [2]:
# Initialize PaddleOCR instance
from sklearn.cluster import KMeans
import json
import numpy as np
from scipy.spatial import distance_matrix
data = json.load(open('output/img_res.json', 'r', encoding='utf-8'))


    


In [3]:
km = KMeans(n_clusters=4, random_state=0)
x_left = [poly[0][0] for poly in data['rec_polys']]
y_left = [poly[0][1] for poly in data['rec_polys']]
x_left = np.array(x_left).reshape(-1, 1)
km.fit_predict(x_left)



array([0, 2, 3, 1, 0, 0, 3, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 2, 0, 1, 2, 3,
       0, 2, 3, 1, 0, 1, 3, 2, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1,
       2, 3, 0, 1, 2, 3, 0, 2, 3, 1, 0, 1, 2, 3, 3, 0, 1, 2, 3, 0, 1, 2,
       3, 0, 1, 2, 3, 0, 1, 2, 3, 3, 0, 1, 2, 2, 3, 1, 3, 0, 1, 2, 3, 0,
       1, 2, 3, 0, 1, 2, 0, 1, 2, 3, 2, 3, 0, 1, 3, 0, 1, 2, 0, 1, 2, 3,
       3, 0, 1, 2, 2, 1, 0, 2, 3, 2, 3, 0, 1, 1, 2, 3, 0, 1, 2, 3, 0, 3,
       0, 1, 2, 3, 0, 1, 1, 2, 3, 0, 1, 3, 0, 1, 2, 3, 0, 2, 3, 1, 0, 1,
       2, 3, 0, 1, 2, 3, 1, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2,
       0, 1, 1, 2, 3, 0, 1, 1, 2, 3, 0, 1, 1, 1, 2, 0, 1, 3, 1, 2, 0, 1,
       2, 0, 1, 1, 2, 3, 0, 1, 1, 2, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3,
       0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0],
      dtype=int32)

In [65]:
text = [[] for _ in range(4)]
boxes = [[] for _ in range(4)]
for idx, j in enumerate(km.labels_):
        text[j].append(data['rec_texts'][idx])
        boxes[j].append(data['rec_polys'][idx])
# order clusters left to right
cluster_centers = km.cluster_centers_.flatten() 
sorted_indices = np.argsort(cluster_centers)
text = [text[i] for i in sorted_indices]
boxes = [boxes[i] for i in sorted_indices]

#sort text and boxed top to bottom within each cluster 
for i in range(4):
    y_coords = [box[0][1] for box in boxes[i]]
    sorted_indices = np.argsort(y_coords)
    text[i] = [text[i][j].strip() for j in sorted_indices]
    boxes[i] = [boxes[i][j] for j in sorted_indices]



In [66]:
import re
lection_number_pattern = re.compile(r'\d.')
for i in range(4):
    for j in range(len(text[i])):
        text[i][j] = re.sub(lection_number_pattern, '', text[i][j]).strip() 

In [67]:

# match text by min vertical distance between their boxes
box_0_centers = [[(b[0][0]+b[3][0])/2, (b[0][1] + b[3][1]) / 2] for b in boxes[0]]
box_1_centers = [[(b[0][0]+b[3][0])/2, (b[0][1] + b[3][1]) / 2] for b in boxes[1]]
box_2_centers = [[(b[0][0]+b[3][0])/2, (b[0][1] + b[3][1]) / 2] for b in boxes[2]]
box_3_centers = [[(b[0][0]+b[3][0])/2, (b[0][1] + b[3][1]) / 2] for b in boxes[3]]

In [77]:
# merge same line elements
def merge(box, text):
    avg_dist = np.median((np.asarray(box)[:,1]-np.roll(np.asarray(box)[:,1], 1))[1:])
    idx = 0
    merged_box = [None for _ in box]
    merged_text = [None for _ in text]
    while idx < len(box):
        b0, t0 = box[idx], text[idx]
        if idx+1 >= len(box) :
            return box, text
        b1, t1 = box[idx+1], text[idx+1]
        
        if np.abs(b0[1]-b1[1]) < avg_dist//4:
            # join left to right
            if b0[0] < b1[0]:
                box[idx] = np.mean([b0, b1],0)
                text[idx] = " ".join([t0,t1])
            else:
                box[idx] = np.mean([b1,b0],0)
                text[idx] = " ".join([t1,t0])
            box.pop(idx+1)
            text.pop(idx+1)
        idx += 1
    
    return box, text

box_0, text0 = merge(box_0_centers.copy(), text[0].copy())
box_1, text1 = merge(box_1_centers.copy(), text[1].copy())
box_2, text2 = merge(box_2_centers.copy(), text[2].copy())
box_3, text3 = merge(box_3_centers.copy(), text[3].copy())

In [78]:
def fifo_match(box0, text0, box1, text1):
    avg_dist = np.median((np.asarray(box0)[:,1]-np.roll(np.asarray(box0)[:,1], 1))[1:])
    
    results = []
    b0, it = box0.pop(0), text0.pop(0)
    b1, de = box1.pop(0), text1.pop(0)
    while True:
        if np.abs(b0[1] - b1[1]) < avg_dist//2:
            results.append([[it],[de]])
            b0, it = box0.pop(0), text0.pop(0)
            b1, de = box1.pop(0), text1.pop(0)

        elif b0[1] < b1[1]:
            results[-1][0].append(it)
            if len(box0) > 0:
                b0, it = box0.pop(0), text0.pop(0)
            else:
                b1, de = box1.pop(0), text1.pop(0)
 
        elif b0[1] > b1[1]:
            results[-1][1].append(de)
            if len(box1) > 0:
                b1, de = box1.pop(0), text1.pop(0)
            else:
                b0, it = box0.pop(0), text0.pop(0)

        if len(box0) == 0 and len(box1) == 0:
            for idx, (it, de) in enumerate(results):
                if len(it) > 1:
                    results[idx][0] = ["<br>".join(it)]
                if len(de) > 1:
                    results[idx][1] = ["<br>".join(de)]
                results[idx][0] = f"\"{results[idx][0][0]}\""
                results[idx][1] = f"\"{results[idx][1][0]}\""
            return results
results0 = fifo_match(box_0, text0, box_1, text1)
results1 = fifo_match(box_2, text2, box_3, text3)
 
            

In [ ]:
class Exporter:
    def __init__(self):
        self.vocabularies = []
    
    def add(self, results):
        [self.vocabularies.append(x) for x in results]
    
    def save(self, filename):
        with open(filename, 'a+') as fout:
            fout.write(f"#deck:{filename}\n")
            for line in self.vocabularies:
                fout.write(f"{line[0]};{line[1]}\n")

export = Exporter()
export.add(results0)
export.add(results1)
export.save("Fantastico! Lezione 0")
    

In [ ]:

[b for b in text[1]]

['ihr; Sie',
 'wer?',
 'ihr seid; Sie sind',
 'wir sind',
 '。',
 'du bist',
 'nein',
 'er',
 'Herr ...',
 'Herr',
 'ja',
 'freut mich',
 'es geht mir gut',
 'in',
 'Mailand',
 'lesen Sie',
 'Dialog',
 'nicht wahr?',
 'oder?',
 'nicht schlecht',
 'nicht',
 'ganz, ziemlich',
 'na ja',
 'wir sind dran',
 'sein',
 'sie ( Person Singular)',
 'wir',
 'sie ( Person Plural)',
 'sie sind',
 '*',
 'aus',
 'Rom',
 'berühmt',
 'Student/in',
 'Professor/in',
 'Florenz',
 'Antwort',
 'Fertig?',
 'Los!',
 'Musik',
 'Vulkan',
 "Giro d'Italia",
 'Lagune',
 'Spaghetti',
 'Pilz',
 'wiederholen Sie',
 'Schokolade',
 'Gardasee',
 'Latte macchiato',
 'Cappuccino',
 'Gondel',
 'Kaffeepause',
 'Pause',
 'Zungenbrecher',
 'Genua',
 'Gin',
 'für',
 'Prost!',
 'Stracciatella-']